In [1]:
# Dogs vs Cats

 # 사전작업

In [29]:
import numpy as np
from glob import glob
import cv2, os, random
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow import keras
from keras.datasets import mnist


from keras.models import load_model
from sklearn.model_selection import train_test_split

In [31]:
seed=2020
np.random.seed(seed)
tf.random.set_seed(seed)

In [32]:
path='dogs-vs-cats/train/train/'

In [33]:
## used for resize and in our model
ROW, COL = 96, 96

dogs, cats = [], []
y_dogs, y_cats = [], []

In [34]:
dog_path=os.path.join(path, 'dog.*')
cat_path=os.path.join(path, 'cat.*')
len(glob(dog_path))

12500

In [42]:
(X_train, y_train0), (X_test, y_test0) = path()
print(X_train.shape, X_train.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test.shape, X_test.dtype)
print(y_test0.shape, y_test0.dtype)

TypeError: 'str' object is not callable

In [36]:
X_train,X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=seed)

In [13]:
classes = ['dog', 'cat']

In [14]:
## just change the labels for 0 and 1
y_dogs=[1 for item in enumerate(dogs)]
y_cats=[0 for item in enumerate(cats)]

In [22]:
x = np.concatenate((dogs,cats), axis=0)
y = np.concatenate((y_dogs, y_cats), axis=0)

In [23]:
x_train,x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=seed)

In [17]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay), 
                 input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
## to save checkpoint to use later
modelpath = "model/dogs_vs_cats-cnn.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [24]:
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_split=0.2,shuffle=True,
          callbacks=[checkpointer, early_stopping_callback])

Train on 1421 samples, validate on 356 samples
Epoch 1/10
1400/1421 [============================>.] - ETA: 0s - loss: 26.8000 - accuracy: 0.4964
Epoch 00001: val_loss improved from inf to 0.69370, saving model to model/dogs_vs_cats-cnn.hdf5
1421/1421 [==============================] - 21s 15ms/sample - loss: 26.4141 - accuracy: 0.4975 - val_loss: 0.6937 - val_accuracy: 0.4831
Epoch 2/10
1400/1421 [============================>.] - ETA: 0s - loss: 0.6930 - accuracy: 0.5014
Epoch 00002: val_loss improved from 0.69370 to 0.69298, saving model to model/dogs_vs_cats-cnn.hdf5
1421/1421 [==============================] - 20s 14ms/sample - loss: 0.6929 - accuracy: 0.5025 - val_loss: 0.6930 - val_accuracy: 0.5393
Epoch 3/10
1400/1421 [============================>.] - ETA: 0s - loss: 0.6934 - accuracy: 0.5314
Epoch 00003: val_loss did not improve from 0.69298
1421/1421 [==============================] - 19s 13ms/sample - loss: 0.6932 - accuracy: 0.5348 - val_loss: 0.6932 - val_accuracy: 0.5140

In [26]:
scores = model.evaluate(X, y, verbose=2)
print('MODEL ACCURACY: %.5f' % scores[1])

2222/2222 - 6s - loss: 0.6447 - accuracy: 0.6017
MODEL ACCURACY: 0.60171
